# Stimuli Variables

In [1]:
import pandas as pd
import numpy as np

In [222]:
# sess_1 = pd.read_csv('1.csv')
# sess_2 = pd.read_csv('2.csv')
#sess_3 = pd.read_csv('3.csv')
#sess_4 = pd.read_csv('4.csv')
#sess_5 = pd.read_csv('5.csv')
#sess_6 = pd.read_csv('6.csv')
#sess_7 = pd.read_csv('7.csv')
#sess_8 = pd.read_csv('8.csv')
sess_9 = pd.read_csv('9.csv')

file = open('../TextFineTuning/lpp_lem.txt', mode='r')
lemmed = file.read()
file.close()

lemmed = lemmed.split()
# -1, 1925, 3986, 6227, 8227, 10079, 12354, 14487 
word_idx = 16347
ite = iter(sess_9.word)
target_word = next(ite)
lemma_toadd = []
categ_toadd = []

while word_idx < len(lemmed) - 1:
    word_idx += 1
    if len(lemmed[word_idx]) < 9:
        continue
    try:
        categ, word = lemmed[word_idx].split('__origin')
        lemma, categ = categ.rsplit('_', 1)
        
        if target_word[0] == '-':
            target_word = target_word[1:]
        if word[0] == '-':
            word = word[1:]
        if word.lower() == target_word.lower():
            lemma_toadd.append(lemma)
            categ_toadd.append(categ)
            try:
                target_word = next(ite)
            except StopIteration:
                break
        elif lemmed[word_idx][:10] == '__punctuat':
            continue
        else:
            print(target_word, lemmed[word_idx:word_idx+10])
    except:
        print(lemmed[word_idx])

il ['chapitre_n__originChapitre', '26_n__origin26', '__PRONOUN____originIl', '__PRONOUN____originy', 'avoir_v__originavait', '__punctuation____origin,', 'à_prep__originà', 'côté_n__origincôté', 'du_det__origindu', 'puits_n__originpuits']
il ['26_n__origin26', '__PRONOUN____originIl', '__PRONOUN____originy', 'avoir_v__originavait', '__punctuation____origin,', 'à_prep__originà', 'côté_n__origincôté', 'du_det__origindu', 'puits_n__originpuits', '__punctuation____origin,']
et ['chapitre_n__originChapitre', '27_n__origin27', '__CCONJ____originEt', 'maintenant_adv__originmaintenant', '__punctuation____origin,', 'bien_adv__originbien', 'sûr_adj__originsûr', '__punctuation____origin,', 'cela_pro__originça', 'faire_v__originfait']
et ['27_n__origin27', '__CCONJ____originEt', 'maintenant_adv__originmaintenant', '__punctuation____origin,', 'bien_adv__originbien', 'sûr_adj__originsûr', '__punctuation____origin,', 'cela_pro__originça', 'faire_v__originfait', 'six_n__originsix']


In [223]:
word_idx

18512

In [75]:
len(lemma_toadd)

1760

In [225]:
sess_9['lemma'] = lemma_toadd
sess_9['categ_alt'] = categ_toadd

In [78]:
sess_1.to_csv('1_lem_auto.csv')
sess_2.to_csv('2_lem_auto.csv')
sess_3.to_csv('3_lem_auto.csv')
sess_4.to_csv('4_lem_auto.csv')
sess_5.to_csv('5_lem_auto.csv')
sess_6.to_csv('6_lem_auto.csv')
sess_7.to_csv('7_lem_auto.csv')
sess_8.to_csv('8_lem_auto.csv')
sess_9.to_csv('9_lem_auto.csv')

# New LPP Vocabulary

In [2]:
lpp_onset = pd.concat([pd.read_csv(str(i) + '_lem_auto.csv') for i in range(1,10)], ignore_index=True)

In [10]:
lpp_onset.groupby(['lemma', 'categ', 'categ_alt', 'Contentword']).count()[['word']].to_csv('lpp_correction.csv')

# Corrected Onset File with WOLF vocabulary

In [132]:
POS_dict = {
    "ADJ": 'a',
    "ADV": 'b',
    "NOM": 'n',
    "VER": 'v'
}

In [133]:
for i in range(1,10):
    session = pd.read_csv(str(i)+'_WOLF.csv', index_col=0)
    session_cw = session.loc[session.proposed_CW=='Y']
    session_cw.loc[:,"proposed_categ"] = session_cw.proposed_categ.map(POS_dict)
    session_cw.loc[:, "word"] = session_cw.proposed_lemma + '_' + session_cw.proposed_categ
    session_cw[['word', 'onset', 'offset']].to_csv('./Context Lemma Onsets/' + str(i)+'_onset_wolf.csv', index=False)

/home/sying/root/anaconda3/envs/micipsa/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


# Generate Embedding Features

In [6]:
ls ../..

Decorrelation/              Similarity-Association-Benchmarks/
LePetitPrince_Pallier2018/  TextFineTuning/
Mango/                      word2vec/
pycortex/                   WordNetEmbeddings/
ROI/


In [7]:
sim_model = pd.read_csv('../../Decorrelation/French_POS/sim_56k_634d.txt', index_col=0, header=None, skiprows=1, sep=' ')

In [2]:
mix_model = pd.read_csv('../../Decorrelation/French_POS/mix_embedding_24519.txt', index_col=0, header=None, skiprows=1, sep=' ')

In [5]:
# Select only NOUNS / VERBS / ADJs

# Configurations
embed_type = 'mix_verb'
model = mix_model
embed_dim = len(model.columns)
embed_voc = len(model)

for i in range(1,10):
    session = pd.read_csv('./Context Lemma Onsets/' + str(i) + '_onset_wolf.csv')
    # session = session.loc[session.word.str.endswith('a') | session.word.str.endswith('b')]
    session = session.loc[session.word.str.endswith('v')]
    session["duration"] = session.offset - session.onset
    features = pd.merge(session, model, left_on='word', right_on=0, how='left')
    print("Session", i, ":", sum(features.isna().any(axis=1)), "words among", len(features), "not found in embedding", embed_type)
    features.dropna(inplace=True)
    for d in range(1, embed_dim+1):
        prefix = str(i) + '_' + embed_type + '_dim' + str(embed_dim) + '_voc' + str(embed_voc) + '_d'
        features[['onset', 'duration', d]].rename(columns={d: 'amplitude'}).to_csv('./Features/' + prefix + str(d) + '.csv', index=False)
    

Session 1 : 9 words among 227 not found in embedding mix_verb
Session 2 : 19 words among 274 not found in embedding mix_verb
Session 3 : 15 words among 313 not found in embedding mix_verb
Session 4 : 17 words among 306 not found in embedding mix_verb
Session 5 : 10 words among 258 not found in embedding mix_verb
Session 6 : 8 words among 296 not found in embedding mix_verb
Session 7 : 9 words among 331 not found in embedding mix_verb
Session 8 : 12 words among 278 not found in embedding mix_verb
Session 9 : 14 words among 335 not found in embedding mix_verb


In [156]:
for i in range(1,10):
    session = pd.read_csv('./Context Lemma Onsets/' + str(i) + '_onset_wolf.csv')
    session["duration"] = session.offset - session.onset
    features = pd.merge(session, sim_model, left_on='word', right_on=0, how='left')
    print("Session", i, ":", sum(features.isna().any(axis=1)), "words among", len(features), "not found in embedding", embed_type)
    features.dropna(inplace=True)
    for d in range(1, embed_dim+1):
        prefix = str(i) + '_' + embed_type + '_dim' + str(embed_dim) + '_voc' + str(embed_voc) + '_d'
        features[['onset', 'duration', d]].rename(columns={d: 'amplitude'}).to_csv('./Features/' + prefix + str(d) + '.csv', index=False)
    

Session 1 : 36 words among 725 not found in embedding sim
Session 2 : 30 words among 812 not found in embedding sim
Session 3 : 32 words among 860 not found in embedding sim
Session 4 : 27 words among 762 not found in embedding sim
Session 5 : 30 words among 732 not found in embedding sim
Session 6 : 33 words among 902 not found in embedding sim
Session 7 : 24 words among 819 not found in embedding sim
Session 8 : 30 words among 712 not found in embedding sim
Session 9 : 27 words among 802 not found in embedding sim


In [172]:
asn_model = pd.read_csv('../../Decorrelation/French_POS/asn_embedding_24519.txt', index_col=0, header=None, skiprows=1, sep=' ')

In [173]:
# Configurations
embed_type = 'asn'
embed_dim = len(asn_model.columns)
embed_voc = len(asn_model)

In [174]:
for i in range(1,10):
    session = pd.read_csv('./Context Lemma Onsets/' + str(i) + '_onset_wolf.csv')
    session["duration"] = session.offset - session.onset
    features = pd.merge(session, asn_model, left_on='word', right_on=0, how='left')
    print("Session", i, ":", sum(features.isna().any(axis=1)), "words among", len(features), "not found in embedding", embed_type)
    features.dropna(inplace=True)
    for d in range(1, embed_dim+1):
        prefix = str(i) + '_' + embed_type + '_dim' + str(embed_dim) + '_voc' + str(embed_voc) + '_d'
        features[['onset', 'duration', d]].rename(columns={d: 'amplitude'}).to_csv('./Features/' + prefix + str(d) + '.csv', index=False)
    

Session 1 : 48 words among 725 not found in embedding asn
Session 2 : 47 words among 812 not found in embedding asn
Session 3 : 38 words among 860 not found in embedding asn
Session 4 : 37 words among 762 not found in embedding asn
Session 5 : 48 words among 732 not found in embedding asn
Session 6 : 60 words among 902 not found in embedding asn
Session 7 : 35 words among 819 not found in embedding asn
Session 8 : 37 words among 712 not found in embedding asn
Session 9 : 41 words among 802 not found in embedding asn


In [164]:
asn_model.index[asn_model.index.duplicated()]

Index(['1_n', 'Monseigneur_n', 'Aaron_n', 'abbé_n', 'abc_n', 'Abel_n',
       'Abel_n', 'Abel_n', 'Abkhazie_n', 'abondance_n',
       ...
       'Étienne_n', 'Étienne_n', 'Étienne_n', 'Ève_n', 'Édimbourg_n',
       'Égypte_n', 'Élisabeth_n', 'Épiaire_n', 'Éthiopie_n', 'Étienne_n'],
      dtype='object', name=0, length=3220)

In [2]:
sig_model = pd.read_csv('../../Decorrelation/French_POS/sig_embedding_24519.txt', index_col=0, header=None, skiprows=1, sep=' ')

In [9]:
# Configurations
embed_type = 'sig'
embed_dim = len(sig_model.columns)
embed_voc = len(sig_model)

In [10]:
for i in range(1,10):
    session = pd.read_csv('./Context Lemma Onsets/' + str(i) + '_onset_wolf.csv')
    session["duration"] = session.offset - session.onset
    features = pd.merge(session, sig_model, left_on='word', right_on=0, how='left')
    print("Session", i, ":", sum(features.isna().any(axis=1)), "words among", len(features), "not found in embedding", embed_type)
    features.dropna(inplace=True)
    for d in range(1, embed_dim+1):
        prefix = str(i) + '_' + embed_type + '_dim' + str(embed_dim) + '_voc' + str(embed_voc) + '_d'
        features[['onset', 'duration', d]].rename(columns={d: 'amplitude'}).to_csv('./Features/' + prefix + str(d) + '.csv', index=False)
    

Session 1 : 48 words among 725 not found in embedding sig
Session 2 : 47 words among 812 not found in embedding sig
Session 3 : 38 words among 860 not found in embedding sig
Session 4 : 37 words among 762 not found in embedding sig
Session 5 : 48 words among 732 not found in embedding sig
Session 6 : 60 words among 902 not found in embedding sig
Session 7 : 35 words among 819 not found in embedding sig
Session 8 : 37 words among 712 not found in embedding sig
Session 9 : 41 words among 802 not found in embedding sig


In [11]:
mix_model = pd.read_csv('../../Decorrelation/French_POS/mix_embedding_24519.txt', index_col=0, header=None, skiprows=1, sep=' ')

In [12]:
# Configurations
embed_type = 'mix'
embed_dim = len(mix_model.columns)
embed_voc = len(mix_model)

In [13]:
for i in range(1,10):
    session = pd.read_csv('./Context Lemma Onsets/' + str(i) + '_onset_wolf.csv')
    session["duration"] = session.offset - session.onset
    features = pd.merge(session, mix_model, left_on='word', right_on=0, how='left')
    print("Session", i, ":", sum(features.isna().any(axis=1)), "words among", len(features), "not found in embedding", embed_type)
    features.dropna(inplace=True)
    for d in range(1, embed_dim+1):
        prefix = str(i) + '_' + embed_type + '_dim' + str(embed_dim) + '_voc' + str(embed_voc) + '_d'
        features[['onset', 'duration', d]].rename(columns={d: 'amplitude'}).to_csv('./Features/' + prefix + str(d) + '.csv', index=False)
    

Session 1 : 48 words among 725 not found in embedding mix
Session 2 : 47 words among 812 not found in embedding mix
Session 3 : 38 words among 860 not found in embedding mix
Session 4 : 37 words among 762 not found in embedding mix
Session 5 : 48 words among 732 not found in embedding mix
Session 6 : 60 words among 902 not found in embedding mix
Session 7 : 35 words among 819 not found in embedding mix
Session 8 : 37 words among 712 not found in embedding mix
Session 9 : 41 words among 802 not found in embedding mix


# Correct Frequency for newly combined words

In [32]:
lexique_db = pd.read_csv('Lexique382.txt', sep='\t')

In [118]:
for i in range(1,10):
    session = pd.read_csv(str(i)+'_WOLF.csv', index_col=0)
    session_fq_cw = pd.merge(session.loc[session.proposed_CW=='Y'], 
             lexique_db[['1_ortho', '3_lemme', '4_cgram', '9_freqfilms2']], 
             left_on=['word', 'proposed_lemma', 'proposed_categ'], 
             right_on=['1_ortho', '3_lemme', '4_cgram'], 
             how='left')[list(session.columns) + ["9_freqfilms2"]]
    session_fq_cw.loc[session_fq_cw['9_freqfilms2'].notna(), 'logfreq'] = session_fq_cw.loc[session_fq_cw['9_freqfilms2'].notna(), '9_freqfilms2'].apply(np.log10)
    session_fq_all = session.loc[session.proposed_CW=='N'].append(session_fq_cw[session_fq_cw.columns[:-1]], ignore_index=True, sort=False)
    session_fq_all.sort_values(by=['onset', 'offset'], inplace=True)
    session_fq_all.to_csv(str(i)+'_WOLF_fqdesamb.csv', index=False)

# Generate Frequency Features

In [5]:
for i in range(1,10):
    session = pd.read_csv('./Word Onsets/' + str(i) + '_WOLF_fqdesamb.csv')
    session["duration"] = session.offset - session.onset
    features = session[["onset", "duration", "logfreq"]]
    prefix = str(i) + '_freq'
    features.rename(columns={'logfreq': 'amplitude'}).to_csv('./Features/' + prefix + '.csv', index=False)

# Generate Word Rate Features

In [8]:
CW_transformation = {
    'Y': 1,
    'N': 0
}

for i in range(1,10):
    session = pd.read_csv('./Word Onsets/' + str(i) + '_WOLF_fqdesamb.csv')
    session["duration"] = session.offset - session.onset
    features = session[["onset", "duration", "proposed_CW"]]
    features['amplitude'] = features['proposed_CW'].map(CW_transformation)
    prefix = str(i) + '_cwrate'
    features[["onset", "duration", "amplitude"]].to_csv('./Features/' + prefix + '.csv', index=False)

/home/sying/root/anaconda3/envs/micipsa/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [9]:
for i in range(1,10):
    session = pd.read_csv('./Word Onsets/' + str(i) + '_WOLF_fqdesamb.csv')
    session["duration"] = session.offset - session.onset
    features = session[["onset", "duration"]]
    features['amplitude'] = 1
    prefix = str(i) + '_wrate'
    features[["onset", "duration", "amplitude"]].to_csv('./Features/' + prefix + '.csv', index=False)

/home/sying/root/anaconda3/envs/micipsa/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
